### 환경설정

In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git 
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

In [ ]:
# !pip install wandb

참조

https://github.com/Beomi/KoAlpaca

https://colab.research.google.com/gist/Beomi/f163a6c04a869d18ee1a025b6d33e6d8/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb

https://wikidocs.net/238524


---

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset

c:\Users\DKSYSTEMS\miniconda3\envs\dseek\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 이름:", torch.cuda.get_device_name(0))

print("Torch CUDA 지원 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("PyTorch 버전:", torch.__version__)

---

### 모델, 토크나이저 로드 및 LoRA 설정, DataLoad

해당 모델은 meta에 키등록을 해야 사용할 수 있습니다.

In [4]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=False)

# tokenizer.pad_token = "<|finetune_right_pad_id|>"
# tokenizer.pad_token_id = 128004
# tokenizer.padding_side = 'right'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_use_double_quant=True, # 이중 양자화(Double Quantization) 적용하여 더 적은 메모리 사용
    bnb_4bit_quant_type="nf4",    # 4-bit 양자화 방식: `nf4` 선택 LLM에 최적화된 새로운 4-bit 방식
    bnb_4bit_compute_dtype=torch.bfloat16, # 연산 시 bfloat16 데이터 타입 사용하여 연산 안정성 증가 ( float16보다 안정적 )
    max_seq_length=512,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"":0},
    # torch_dtype=torch.bfloat16, #양자화 안할 거면 사용
    quantization_config=bnb_config, #양자화 할거면 사용
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = "<|finetune_right_pad_id|>"
tokenizer.pad_token_id = 128004
tokenizer.padding_side = 'right'

print(model)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.40s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

샘플이 너무 적으면 과적합(Overfitting) 위험이 크므로 데이터를 증강(Augmentation)하는 것이 필요.

In [5]:
from datasets import load_dataset

# LLaMA 3 Chat 템플릿 적용
chat_template = """<|begin_of_text|><|start_header_id|>지시사항<|end_header_id|>
{SYSTEM}<|eot_id|><|start_header_id|>입력<|end_header_id|>
{INPUT}<|eot_id|><|start_header_id|>응답<|end_header_id|>
{OUTPUT}<|eot_id|>"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    
    for instruction, output in zip(instructions, outputs):
        text = chat_template.format(
            SYSTEM="아래는 작업을 설명하는 지시사항입니다. 입력된 내용을 바탕으로 적절한 응답을 작성하세요.",
            INPUT= instruction,
            OUTPUT= output 
        ) 
        
        texts.append(text)
    
    return {"text": texts}

# 데이터셋 로드
dataset = load_dataset("json", data_files="../00_Data/KoAlpaca_train.json")
# 데이터셋 변환
dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=['instruction', 'output'])

split_data = dataset['train'].train_test_split(test_size=0.05, seed=42)
train_data, val_data = split_data['train'], split_data['test']


Generating train split: 40 examples [00:00, 1399.05 examples/s]
Map: 100%|██████████| 40/40 [00:00<00:00, 5308.24 examples/s]


In [6]:
df = train_data.to_pandas()  # "train" 데이터셋을 pandas로 변환
print(df.head())  # 첫 5개 샘플 확인

                                                 url  \
0  https://kin.naver.com/qna/detail.naver?d1id=8&...   
1  https://kin.naver.com/qna/detail.naver?d1id=11...   
2  https://kin.naver.com/qna/detail.naver?d1id=11...   
3  https://kin.naver.com/qna/detail.naver?d1id=7&...   
4  https://kin.naver.com/qna/detail.naver?d1id=6&...   

                                                text  
0  <|begin_of_text|><|start_header_id|>지시사항<|end_...  
1  <|begin_of_text|><|start_header_id|>지시사항<|end_...  
2  <|begin_of_text|><|start_header_id|>지시사항<|end_...  
3  <|begin_of_text|><|start_header_id|>지시사항<|end_...  
4  <|begin_of_text|><|start_header_id|>지시사항<|end_...  


In [7]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()#훈련 시 메모리 절약 (출력값을 필요할 때만 계산)
model = prepare_model_for_kbit_training(model)

In [8]:
#LoRA(PEFT) 설정을 적용하여 기존 모델을 효율적으로 미세 조정
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",# LoRA를 적용할 작업 유형 (CAUSAL_LM: 언어 모델)
    r=8,# LoRA 랭크 (적은 수록 가벼움, 크면 성능 향상 가능)
    lora_alpha=16,   # 일반적으로 LoRA의 효과를 조절하는 파라미터 (값이 크면 LoRA 가중치의 영향 증가)
    lora_dropout=0.05, # Dropout 확률 (일반적으로 0~0.1 추천)
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)

---

### Finetuning

In [10]:
#하이퍼 파라미터 설정

epoch = 2 # 전체 데이터셋을 몇 번 반복해서 학습할 것인지
batch_size =4
gradient_step =2
learningrate = 1e-3 # 1e-3 ~ 1e-6 가 일반적인 러닝 레이트 범위 ( 1e-4 에서 시작하는거 추천 )
# 1e-3 (0.001) → 매우 높은 학습률, 빠른 학습 가능하지만 불안정할 수도 있음
# 5e-4 (0.0005) → 비교적 빠른 학습, 안정성도 고려한 값
# 1e-4 (0.0001) → 일반적으로 많이 사용되는 기본값
# 5e-5 (0.00005) → 안정성과 학습 속도 균형이 적절한 값
# 1e-5 (0.00001) → 비교적 낮은 학습률, 정밀한 파인튜닝에 적합
# 5e-6 (0.000005) → 매우 낮은 학습률, 기존 모델을 크게 변경하지 않으면서 미세 조정할 때 유용
# 1e-6 (0.000001) → 극도로 낮은 학습률, 작은 변화만 필요할 때 사용
step = 300 # 최대 학습 스텝

outName = f"{model_id.split('/')[-1]}-{epoch}-{batch_size}-{gradient_step}-{learningrate}"
output_dir = f"../01_Models/01_RoLaModels/{outName}"

print(outName)
print(output_dir)

Llama-3.1-8B-Instruct-2-4-2-0.001
./01_Models/01_RoLaModels/Llama-3.1-8B-Instruct-2-4-2-0.001


In [ ]:
# wanDB 사용할거면 실행
import wandb
wandb.login(key="")

In [11]:
import transformers
import wandb
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer

train_args = TrainingArguments(
    # max_seq_length=512,
    # dataset_text_field="text",
    # packing=False,

    per_device_train_batch_size=batch_size, # 배치 크기 (GPU당 샘플 개수)
    gradient_accumulation_steps=gradient_step,  # 메모리 최적화 Gradient Accumulation 누적 스텝 (메모리 부족 시 증가 가능)
    gradient_checkpointing=True, # 활성화하면 GPU 메모리 사용 감소 가능

    num_train_epochs=epoch, 
    # max_steps=step ,  

    optim="adamw_torch", # paged_adamw_8bit (VRAM절약 성능 하락) adamw_torch(정확도 높음 메모리사용량 높음)

    learning_rate=learningrate,  # 학습률 (기본 2e-4)
    lr_scheduler_type="linear", # 학습률 스케줄러 종류 ( linear, cosine, constant )

    fp16=True, #정밀도
    bf16=False, # 이거 사용하면 오류남 왜인진 모르겠음;; 아마  (A100 GPU같이 고성능 gpu에서 쓰이면 좋아서 그런듯 후...)
   
    weight_decay=0.01, # 모델이 과적합(Overfitting)되는 것을 방지하기 위해 가중치(Weight)에 패널티를 부여하는 기법  ( 일반화 성능 향상 )

    warmup_ratio=0.1, # 상승곡선

    seed=42,
    
    evaluation_strategy="steps", # eval_steps마다 평가
    eval_steps=5, # eval 훈련 스텝이 xx번 진행될 때마다 검증 데이터셋 평가

    logging_steps=2,
    output_dir= output_dir,
    save_strategy="epoch",
    log_level="debug",

    report_to="wandb", # none, wandb 사용  
)

# Trainer Setup
trainer = SFTTrainer(
    model=model,
    peft_config=lora_config,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=train_args,
)

# wandb 사용할거면 주석풀고 사용하세요. 
# wandb.finish() # 이전 실행 종료 (안 하면 새로운 실행이 안 생길 수도 있음)

# # wandb 사용
# wandb.init(
#     project="exaone-learning_rate",
#     name=outName,
#     reinit=True,
#     config={
#         "learning_rate": learningrate,
#         "batch_size": batch_size,
#         "gradient_accumulation_steps": gradient_step,
#         "num_train_epochs": epoch
#     }
# )

model.config.use_cache = False
# 학습 시작
trainer.train()

c:\Users\DKSYSTEMS\miniconda3\envs\dseek\lib\site-packages\transformers\training_args.py:1609: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\DKSYSTEMS\AppData\Local\Temp\ipykernel_26560\228844736.py:44: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Truncating eval dataset: 100%|██████████| 2/2 [00:00<00:00, 271.25 examples/s]
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Currently training with a batch size of: 4
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.f

Step,Training Loss,Validation Loss
5,2.326800,1.671333
10,1.494800,1.456326


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: url, text. If url, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2
  Batch size = 8
Saving model checkpoint to ./01_Models/01_RoLaModels/Llama-3.1-8B-Instruct-2-4-2-0.001\checkpoint-5
loading configuration file config.json from cache at C:\Users\DKSYSTEMS\.cache\huggingface\hub\models--meta-llama--Llama-3.1-8B-Instruct\snapshots\0e9e39f249a16976918f6564b8830bc894c89659\config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "

TrainOutput(global_step=10, training_loss=2.0066962242126465, metrics={'train_runtime': 53.6433, 'train_samples_per_second': 1.417, 'train_steps_per_second': 0.186, 'total_flos': 1340934079905792.0, 'train_loss': 2.0066962242126465})

In [12]:
model.eval() # 모델의 가중치는 변경하지 않고, forward 연산만 수행함.
model.config.use_cache = True  # 이전 계산 결과를 저장하고 사용	추론 속도 빨라짐, 메모리 사용 증가

---

RoLA모델 저장

In [15]:
# Save Model
saveLoRA_dir = f"{output_dir}/LoRA"

trainer.save_model(saveLoRA_dir)
print(f"Model saved at {saveLoRA_dir}")

# Save Model HugginFace
# model.save_pretrained(save_dir)
# tokenizer.save_pretrained(save_dir)

Saving model checkpoint to ./01_Models/01_RoLaModels/Llama-3.1-8B-Instruct-2-4-2-0.001/LoRA
loading configuration file config.json from cache at C:\Users\DKSYSTEMS\.cache\huggingface\hub\models--meta-llama--Llama-3.1-8B-Instruct\snapshots\0e9e39f249a16976918f6564b8830bc894c89659\config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope

Model saved at ./01_Models/01_RoLaModels/Llama-3.1-8B-Instruct-2-4-2-0.001/LoRA


---

### 테스트

In [13]:
# 채팅 스타일 프롬프트 (Llama-3의 Chat 모델용)
chat_prompt = """<|begin_of_text|><|start_header_id|>지시사항<|end_header_id|>
아래는 작업을 설명하는 지시사항입니다. 입력된 내용을 바탕으로 적절한 응답을 작성하세요.<|eot_id|>
<|start_header_id|>입력<|end_header_id|>
다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?<|eot_id|>
<|start_header_id|>응답<|end_header_id|>
"""
# 토큰화 및 모델 실행
input_ids = tokenizer(chat_prompt, return_tensors="pt").input_ids.to("cuda")
with torch.no_grad():
    output_ids = model.generate(input_ids, max_new_tokens=100, temperature=0.7, top_p=0.9, do_sample=True)

# 출력 변환
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("LLM 응답:", output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


LLM 응답: 지시사항
아래는 작업을 설명하는 지시사항입니다. 입력된 내용을 바탕으로 적절한 응답을 작성하세요.
입력
다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?
응답
'다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있어요. 'Madam, I'm Adam'이 대표적인 예시입니다. 또한, 다른 나라의 언어에도 이러한 문장이 있습니다. '아래는 몇 가지 예시입니다.


---

###  기존 모델과 로라 병합 (어뎁터 유지) 풀파인튜닝 저장 

merge_and_unload 양자화 상태로 해버리면 로라 어뎁터가 망가져버림 그래서 양자화 하지 않고 저장해줘야함

In [16]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.1-8B-Instruct"
#로라 모델경로를 확인해봐야 해요
peft_model_id = saveLoRA_dir

loadModel = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # float16로 유지 bfloat16
    device_map="auto"
)

loadModel = PeftModel.from_pretrained(loadModel, peft_model_id, device_map="auto")
loadtokenizer = AutoTokenizer.from_pretrained(model_id)


loading configuration file config.json from cache at C:\Users\DKSYSTEMS\.cache\huggingface\hub\models--meta-llama--Llama-3.1-8B-Instruct\snapshots\0e9e39f249a16976918f6564b8830bc894c89659\config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta":

In [17]:
loadModel = loadModel.merge_and_unload() #실제 병합
merged_model_path = "../01_Models/02_FullFinetuningModels"
loadModel.save_pretrained(merged_model_path)
loadtokenizer.save_pretrained(merged_model_path)

Configuration saved in ../01_Models/02_FullFinetuningModels\config.json
Configuration saved in ../01_Models/02_FullFinetuningModels\generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 4 checkpoint shards. You can find where each parameters has been saved in the index located at ../01_Models/02_FullFinetuningModels\model.safetensors.index.json.
tokenizer config file saved in ../01_Models/02_FullFinetuningModels\tokenizer_config.json
Special tokens file saved in ../01_Models/02_FullFinetuningModels\special_tokens_map.json


('../01_Models/02_FullFinetuningModels\\tokenizer_config.json',
 '../01_Models/02_FullFinetuningModels\\special_tokens_map.json',
 '../01_Models/02_FullFinetuningModels\\tokenizer.json')

---

### GGUF  llama cpp 사용

참조

https://m.blog.naver.com/112fkdldjs/223513042256

https://github.com/ollama/ollama/issues/4442

https://github.com/ollama/ollama/issues/4572

https://github.com/teddylee777/langserve_ollama/tree/main/ollama-modelfile/Llama-3-8B-Instruct

Q8_0: 8비트 양자화 모델로, 원본 모델의 품질을 거의 그대로 유지하면서 크기를 절반으로 줄입니다.

Q6_K, Q5_K_M, Q5_K_S: 6비트와 5비트 양자화 모델들로, 품질 손실은 미미하지만 크기가 더 작습니다.

Q4_K_M, Q4_K_S: 4비트 양자화 모델로, 약간의 품질 손실이 있지만 크기가 매우 작습니다.

In [ ]:
!python ../llama.cpp/convert_hf_to_gguf.py ../01_Models/02_FullFinetuningModels --outtype q8_0